In [29]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset
from d2l import torch as d2l

In [30]:
'''
生成数据
'''
def synthetic_data(w,b,num_example):
    X=torch.normal(0,1,(num_example,len(w)))
    y=torch.matmul(X,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [31]:
'''
用迭代器封装随机生成的数据
'''
def load_array(data_arrays,batch_size,is_train=True):
    dataset=TensorDataset(*data_arrays) # TensorDataset接受特征和标签并封装为Dataset
    return DataLoader(dataset,batch_size=batch_size,shuffle=is_train)

In [32]:
batch_size=10
data_iter=load_array((features,labels),batch_size)

In [33]:
# 观察是否正常工作
next(iter(data_iter))

[tensor([[-0.4564, -0.8860],
         [-0.3930,  1.4651],
         [ 0.9992,  1.0779],
         [-0.5995, -1.1271],
         [ 0.7299, -0.5281],
         [ 0.9507,  1.7498],
         [ 0.5979, -1.5631],
         [ 0.5104,  1.9488],
         [-0.6493,  1.0971],
         [-1.2549,  0.1513]]),
 tensor([[ 6.3150],
         [-1.5595],
         [ 2.5221],
         [ 6.8380],
         [ 7.4642],
         [ 0.1455],
         [10.7019],
         [-1.4121],
         [-0.8180],
         [ 1.1631]])]

In [34]:
from torch import nn
# 定义网络
net=nn.Sequential(nn.Linear(2,1))

In [35]:
# 初始化网络参数
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [36]:
# 损失函数
loss=nn.MSELoss()

In [37]:
# 优化器
trainer=torch.optim.SGD(net.parameters(),lr=0.01)

In [38]:
num_epoch=3
for epoch in range(num_epoch):
    for X,y in data_iter:
        l=loss(net(X),y)
        # 优化器每轮清空参数
        trainer.zero_grad()
        l.backward()
        trainer.step()
    with torch.no_grad():
        l=loss(net(features),labels)
        print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.660061
epoch 2, loss 0.014986
epoch 3, loss 0.000470


In [39]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0070, -0.0172])
b的估计误差： tensor([0.0082])
